## Google's PageRank algorithm

Reference: 
- [Wikipedia article on PageRank](https://en.wikipedia.org/wiki/PageRank)
- Original PageRank paper: Page, Lawrence and Brin, Sergey and Motwani, Rajeev and Winograd, Terry (1999) [The PageRank Citation Ranking: Bringing Order to the Web](http://ilpubs.stanford.edu:8090/422/1/1999-66.pdf). Technical Report. Stanford InfoLab.

### PageRank
Q: For any search engine (e.g., chrome), if a user searches a term, how does the search engine decide the order of web pages to display?

- The frequency of a term appears in the webpage. Doesn't usually work well, because it's vulnerable to "Google bombing" (repeating the term a lot in a page in the hopes that it'll go up in the rankings)
- **Popularity**: the numbers of links to that page
- **Quality**: the quality of the links to that page
> Being linked from ten low-follower blog post is not the same as getting one link from the front page of Wikipedia.

<img src="./PR.png"> # width="400" align=left

Larry **Page** and Sergey Brin set out to create the **PageRank algorithm**for ranking of websites for their search engine:
- [wikipedia definition](https://en.wikipedia.org/wiki/PageRank) 
- considers the ranking of webpage based on the number and quality of links between pages 
- computes the probability that a person will end up in a given page


Before we can talk about the algorithm, we need to come up with a **model** that allows us to reason about this problem. What are we concerned with?

- Websites
- Links between websites

We can **model this as a graph**:
- Review: vertex, edge, in-degree, out-degree
- Our **goal** is to compute the probability that a page will be visited, based on the structure of the graph.

<img src="web-graph.png" width="400" align=left>, <img src="graph.png" width="200" align=right>

## Loading the data

The data is provided in a text file (i.e., tiny.txt) with information about the number of pages and the links between the pages. The graph we saw earlier would be represented by the following file:

    5
    0 1
    1 2
    1 2
    1 3
    1 3
    1 4
    2 3
    3 0
    4 0
    4 2

**Q**: How do we represent the graph internally?

Use an adjacency matrix: the value at location (i, j) is the number of links from page i to page j.

<img src="./graph.png" width="200" align=left>

In [1]:
from pagerank import read_graph
page_links, out_degree = read_graph("tiny.txt")

In [2]:
page_links # the value at location (i, j) is the number of links from page i to page j

[[0, 1, 0, 0, 0],
 [0, 0, 2, 2, 1],
 [0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0],
 [1, 0, 1, 0, 0]]

In [3]:
out_degree # row summation

[1, 5, 1, 1, 2]

## Computing the transition matrix

Our goal is to compute the page popularity (i.e., probability that a page will be visited), based on the structure of the graph. There are multiple ways of doing this. One way is to simulate the behaviour of a **typical user** browsing the web. We will use a crude but useful model of a "random surfer":

- The user starts at a given page, and then choose the next page as follows:
  - 90% of the time, the user will follow a link on the current page (the link is chosen randomly, but with probability proportional to the ratio of outgoing links)
  - 10% of the time, the user will just go to a random page (regardless of whether it is linked from the current page)

<img src="./graph.png" width="100" align=left>

For example, the user starts at page 0, the probability of this user moves to page 1:
- link probability: 0.9 * (the number of links from 0 to 1/the total number of links goes out from 0) 
> 0.9 * (1/1) = 0.9 <br>

- leap probability: 0.1 * (1/the total number of pages)
> 0.1 * (1/5) = 0.02

<img src="./PR_graph.png" width="400" align=left>

<img src="./PR_prob.png" width="800" align=left>

Compute a **transition matrix** to record the probability of going from page i to page j.

In [4]:
def compute_transition(out_degree):
    """
    Initialize a transition matrix with 0.0 probability
    """
    n = len(out_degree)

    # Create a n x n "matrix"(using lists of lists)
    transition_matrix = []
    for i in range(n):
        transition_matrix.append([0.0]*n) # initialize all probabilities to 0.0
            
    return transition_matrix

Note: Working with matrices (represented as lists of lists) is a bit messier than working with lists (in particular, creating an empty matrix requires using a for loop, and accessing matrices is typically done with index variables). Later in the quarter, we'll see a library called **NumPy** that allows us to work with matrices more efficiently and elegantly.

In [5]:
compute_transition(out_degree) # Initialize the transition probabilities

[[0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0]]

Compute the probability from page i to page j:
- 90% of the times, the user goes to page "j" with a probability proportional to the number of links from "i" to "j": ``0.9 * (count(i,j)/out_degree of i)``
- 10% of the times, the user just randomly pick any page: ``0.1 * (1/n)``

In [6]:
def compute_transition(page_links, out_degree):
    """
    Revise the function:
        - Initialize the transition matrix with 0.0 probabilities
        - Calculate the real probability from page i to page j
    """
    n = len(out_degree)

    # Create an empty n x n "matrix" (using lists of lists)
    transition_matrix = []
    for i in range(n):
        transition_matrix.append([0.0]*n)   

    # Iterate over every location (i,j) and fill in the probability from page i to page j
    for i in range(n):
        for j in range(n):
            transition_matrix[i][j] = (0.9 * page_links[i][j]/out_degree[i]) + (0.1 * 1/n)

    return transition_matrix

In [7]:
transition_matrix = compute_transition(page_links, out_degree)

In [8]:
transition_matrix

[[0.02, 0.92, 0.02, 0.02, 0.02],
 [0.02, 0.02, 0.38, 0.38, 0.19999999999999998],
 [0.02, 0.02, 0.02, 0.92, 0.02],
 [0.92, 0.02, 0.02, 0.02, 0.02],
 [0.47000000000000003, 0.02, 0.47000000000000003, 0.02, 0.02]]

## Simulating the random surfer (i.e., the user that browses the web)

**Q**: How do we figure out what the next page is?

Let's say we're in page 4:

In [9]:
transition_matrix[4]

[0.47000000000000003, 0.02, 0.47000000000000003, 0.02, 0.02]

We need to choose a page according to the given distribution for that page (i.e., page 0 with probability 0.47, page 1 with probability 0.02, etc.)

**Q**: Surfer is on page i, how to choose the next page j?
- Row i in the transition matrix shows the probability from page i to other pages.
- Compute the cumulative probabilities for row i.
- Generate a random number between 0.0 and 1.0.
- Choose page j based on the interval where r lies (i.e., we pick the first cumulative probability that is greater than r)

<img src="./random_walker.png" width="800" align=left>

If the user is now at page 4, how to determine the next page? 

In [10]:
import random

probabilities = transition_matrix[4]

n = len(transition_matrix) # the total number of web pages
r = random.uniform(0.0, 1.0) # randomly generate a prob and choose the next page based on it
psum = 0.0 # the cumulative sum

for j in range(n): # iterate over all the pages
    psum += probabilities[j]
    if psum >= r: # stops when the cumulative probability is greater than r
        print(j)
        break

0


In [11]:
# Randomly generate a value in the uniform distribution
r = random.uniform(0.1, 1.0)
r

0.734781926543613

Let's put this into a function:

In [12]:
def make_one_move(transition_matrix, current_page):
    """
    The user is at current page, decide the next page to go based on the transition matrix
    """
    n = len(transition_matrix) # the total number of web pages
    r = random.uniform(0.0, 1.0)
    psum = 0.0

    for j in range(n):
        psum += transition_matrix[current_page][j]
        if psum >= r:
            return j

The user is currently at page 4 and which is the most likely page to go? We **simulate this process 1000 times** and keep a count of how many times each page is the next page to move to. Then, we can check the distribution of the possible next pages, which gives us a rough approximation of the possibility that an arbitrary user would visit that page.

In [13]:
num_simulation = 1000

nextpage_visits = [0]*len(transition_matrix) # record the frequency of a page being visited

current_page = 4
for i in range(num_simulation):
    next_page = make_one_move(transition_matrix, current_page)
    nextpage_visits[next_page] += 1 

In [14]:
for i, count in enumerate(nextpage_visits):
    print(i, count)

0 478
1 15
2 462
3 29
4 16


<img src="./graph.png" width="100" align=left>

If the user starts on page 0, and then **takes 1000 steps of move**, what is the frequency that each page is being visited during this process?

In [21]:
num_steps = 1000

transition_matrix = compute_transition(page_links, out_degree)
n = len(transition_matrix)
times_visited = [0]*n # record the frequency of a page being visited

page = 0 # start from page 0
for t in range(num_steps):
    page = make_one_move(transition_matrix, page) # update the page after every move
    times_visited[page] += 1

In [22]:
# For the 1000 steps of move, the distribution of pages being visted
for i, count in enumerate(times_visited):
    print(i, count)

0 269
1 260
2 151
3 244
4 76


Now let's put this in a function and have it print the normalized counts:

In [37]:
def random_surfer(transition_matrix, num_steps):
    """
    Simulate the behavior of a random surfer
    """
    
    n = len(transition_matrix)
    page = 0 # start from page 0
    times_visited = [0]*n

    # pages being visited in num_steps of move
    for t in range(num_steps):
        page = make_one_move(transition_matrix, page) # the page get updated
        times_visited[page] += 1

    # normalization: change count to probability 
    for i, count in enumerate(times_visited):
        v = count / num_steps
        print(i, v)

In [32]:
# the rank of the page popularity during 1000 steps of move: 
random_surfer(transition_matrix, 1000)

0 0.272
1 0.268
2 0.146
3 0.242
4 0.072


### Popularities match with the graph
Page 3 has three inbound links (popular). <br>
Page 0 has two inbound links, and one link is from the popular page 3. <br>
Page 1 has one inbound link from the popular page 0. <br>

Page 2 has three inbound links, but neither page 1 nor page 4 is popular. <br>
Page 4 has one non-popular inbound link from page 1.

<img src="./graph.png" width="200" align=left>

## Computational Thinking:
- Goal: rank pages based on the popularity
- Decomposition/abstraction
> Popularity: the probability that a page will be visited<br>
> compute popularity based on the structure of the graph. <br>
- Modeling:
> Graph representation of the data <br>
> links: nodes <br>
> connections between links: edges<br>
> similate the random surfer: 90%: 10% <br>
- Algorithms:

The pseudocode:
    
    page_links, out_degree = Load data
    transition_matrix = <compute transition matrix>
    times_visited = <list of size N>
    for i in range(num_steps):
        page = <take a step with the random surfer>
        times_visited[page] += 1
    
    for c in times_visited:
        print c / num_steps

- Complexity:
> matrix representation <br>
> iterate over and fill in every cell in the transition matrix <br>
> the frequency a node is visited during random walk <br>
> the number of moves for random walk <br>
- Divide each part into a specific function. 
> compute_transition <br>
> make_one_move <br>
> random_surfer <br>

In [34]:
from pagerank import read_graph

counts, out_degree = read_graph("tiny.txt")
transition_matrix = compute_transition(page_links, out_degree)
random_surfer(transition_matrix, 1000)

0 0.27
1 0.274
2 0.14
3 0.248
4 0.068


In [35]:
page_links, out_degree = read_graph("medium.txt")
transition_matrix = compute_transition(page_links, out_degree)
random_surfer(transition_matrix, 1000)

0 0.002
1 0.012
2 0.009
3 0.002
4 0.01
5 0.011
6 0.067
7 0.029
8 0.011
9 0.031
10 0.002
11 0.029
12 0.009
13 0.042
14 0.02
15 0.022
16 0.014
17 0.006
18 0.033
19 0.026
20 0.004
21 0.035
22 0.059
23 0.045
24 0.007
25 0.001
26 0.0
27 0.011
28 0.055
29 0.001
30 0.039
31 0.029
32 0.02
33 0.011
34 0.021
35 0.019
36 0.005
37 0.032
38 0.009
39 0.026
40 0.019
41 0.029
42 0.024
43 0.012
44 0.036
45 0.001
46 0.014
47 0.021
48 0.017
49 0.011
